<a href="https://colab.research.google.com/github/WilliamPoe/CSCI-290/blob/main/notebooks/k_means_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [42]:
class KMeans_clustering:

    def __init__(self, k=3, max_iterations=100):
        self.k = k
        self.max_iterations = max_iterations

    def euclidean_distance(self, point1, point2): # Calculate the euclidean distance
        return np.sqrt(np.sum((np.array(point1) - np.array(point2))**2))

    def ran_centroids(self, data): # Picks random centroids from dataset
        ran_centroids = data.sample(self.k).index
        return data.loc[ran_centroids]

    def calc_centroids(self, clusters): # Calculate the new centroids from the clusters
        centroids = np.empty((self.k, self.n_features))
        for i, cluster in enumerate(clusters):
            centroids[i] = np.mean(cluster, axis=0)
        return centroids

    def distances(self, data, centroids): # Calculate the distance between the data and the centroids using euclidean distances function
        print('This is the data:',data)
        print('This is the centroids:', centroids)
        distances = [self.euclidean_distance(data, centroid) for centroid in centroids]
        return np.argmin(distances)

    def clustering(self, data, centroids): # Clusters the data together based off of the calculated distances
        clusters = [[] for _ in range(self.k)]
        labels = np.empty(self.n_samples)
        for i, point in data.iterrows():
            print('This is the point:', point)
            print('This is the i:',i)
            cluster_index = self.distances(point, centroids)
            clusters[cluster_index].append(point)
            labels[i] = cluster_index
        return clusters, labels

    def fit(self, data): # Fits the data
      self.n_samples, self.n_features = data.shape
      self.centroids = self.ran_centroids(data)

      for _ in range(self.max_iterations):
        self.clusters, self.labels = self.clustering(data, self.centroids)
        cur_centroids = self.calc_centroids(self.clusters, data)
        if np.all(self.centroids == cur_centroids):
          break
        self.centroids = cur_centroids

In [43]:
df = pd.read_csv('https://github.com/WilliamPoe/CSCI-290/raw/refs/heads/main/Data/penguins.csv')

def plot_clusters(data, labels, centroids):
    plt.figure(figsize=(5, 3))

    # Plots the data points with cluster colors
    plt.scatter(data.iloc[:, 0], data.iloc[:, 1], c=labels, cmap='viridis', s=50)

    # Plots the centroids with a different color
    plt.scatter(centroids.iloc[:, 0], centroids.iloc[:, 1], c='red', marker='x', s=200)

    plt.title('Cluster Visualization')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')

    plt.show()

numeric_df = df.select_dtypes(include=[np.number])


#for attrib in df.columns:
#    # Checks if the current attrib is 'object'
#    if df[attrib].dtype in ['object']:
#      # Converts the attrib in both the data set and new_data to numerical data
#      df[attrib] = df[attrib].fillna('Unknown')
#      df[attrib] = df[attrib].astype('category').cat.codes
numeric_df = numeric_df.drop(columns='year')
numeric_df = numeric_df.dropna()
print(numeric_df.dtypes)


model = KMeans_clustering(k=3)
model.fit(numeric_df)
labels = model.labels
centroids = model.centroids

plot_clusters(numeric_df, labels, centroids)


bill_length_mm       float64
bill_depth_mm        float64
flipper_length_mm    float64
body_mass_g          float64
dtype: object
This is the point: bill_length_mm         39.1
bill_depth_mm          18.7
flipper_length_mm     181.0
body_mass_g          3750.0
Name: 0, dtype: float64
This is the i: 0
This is the data: bill_length_mm         39.1
bill_depth_mm          18.7
flipper_length_mm     181.0
body_mass_g          3750.0
Name: 0, dtype: float64
This is the centroids:      bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g
209            45.5           15.0              220.0       5000.0
71             39.7           18.4              190.0       3900.0
269            48.8           16.2              222.0       6000.0


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('float64'), dtype('<U14')) -> None

In [26]:
def euclidean_distance(point1, point2): # Calculate the euclidean distance
        return np.sqrt(np.sum((np.array(point1) - np.array(point2))**2))

In [25]:
euclidean_distance(numeric_df.iloc[1], numeric_df.iloc[5])

175.07289910205978

In [27]:
euclidean_distance(numeric_df.iloc[1], numeric_df.iloc[5])

175.07289910205978

In [32]:
numeric_df

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,39.1,18.7,181.0,3750.0
1,39.5,17.4,186.0,3800.0
2,40.3,18.0,195.0,3250.0
4,36.7,19.3,193.0,3450.0
5,39.3,20.6,190.0,3650.0
...,...,...,...,...
339,55.8,19.8,207.0,4000.0
340,43.5,18.1,202.0,3400.0
341,49.6,18.2,193.0,3775.0
342,50.8,19.0,210.0,4100.0
